## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0407/review_data(seg+pos+stopwords)_train_n+v+f+p.csv'
df_train = pd.read_csv(path)
path = '../data/0407/review_data(seg+pos+stopwords)_test_n+v+f+p.csv'
df_test = pd.read_csv(path)

### 檢查重複值、空值

In [3]:
#印出重複資料
print(df_test[df_test.duplicated()])
print(df_train[df_train.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []
Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [4]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [5]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [6]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [7]:
df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)

C:\Users\g3190\AppData\Local\Temp/ipykernel_15384/3099386144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value.rename(columns={'value': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_15384/3099386144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_15384/3099386144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

### 清理資料(移除詞性標註的文字)

In [8]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [9]:
#訓練集
df_value_train['filtered_word'] = df_value_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_comfort_train['filtered_word'] = df_comfort_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_location_train['filtered_word'] = df_location_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_cleanliness_train['filtered_word'] = df_cleanliness_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_service_train['filtered_word'] = df_service_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_facilities_train['filtered_word'] = df_facilities_train.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
#測試集
df_value_test['filtered_word'] = df_value_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_comfort_test['filtered_word'] = df_comfort_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_location_test['filtered_word'] = df_location_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_cleanliness_test['filtered_word'] = df_cleanliness_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_service_test['filtered_word'] = df_service_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)
df_facilities_test['filtered_word'] = df_facilities_test.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [10]:
df_value_test

,label,filtered_word
0,0.0,廚房 提供 餅乾 飲料 買 全 家 買
1,0.0,離 車站 客運站 機會 想 回住
2,0.0,離 車站 房間 訂放 人 接洽 主題 民宿 推薦 洽公 背包客 住宿
3,0.0,人 房 空間 床埔 整體 乾淨 舒適
4,1.0,價格 公道 背包客 民宿 老闆 客氣 旁邊 機車 代步 工具 租
...,...,...
415,0.0,房間 看到 海馬哥 熱情 民宿 風格 簡約 ikea 東西
416,0.0,老闆娘 說 問 嬰兒 小推車 小孩 吃 烤 肉
417,0.0,乾淨 舒適 闆娘 小 幫手 熱情 推薦 景點 美食 招待 水果 甜 好好 吃 完食
418,0.0,老闆娘 親切 贈予 水果 點心 房間 舒適 整潔 小孩 喜歡


## 模型架構

### 套件引用

In [10]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [11]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

### 切分數據label

In [12]:
def split_label(df_train, df_test):
    X_train = df_train.filtered_word.tolist()
    y_train = df_train.label
    X_test = df_test.filtered_word.tolist()
    y_test = df_test.label
    return X_train, y_train, X_test, y_test

### SVM模型設計

#### (1) baseline

In [13]:
def SVM_model(df_train, df_test):
    print("SVM baseline")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (2)執行採樣 => 解決類別不平衡

In [14]:
def SVM_model2(df_train, df_test):
    print("ADASYN")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [15]:
def SVM_model3(df_train, df_test):
    print("SMOTE")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [16]:
def SVM_model4(df_train, df_test):
    print("RandomOverSampler")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [17]:
def SVM_model5(df_train, df_test):
    print("RandomUnderSampler")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

### Ensemble 模型

In [31]:
def AdaBoost_model(df_train, df_test):
    print("AdaBoost_model")
    #切分數據集
    X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 模型 facilities

In [32]:
SVM_model(df_facilities_train, df_facilities_test)
SVM_model2(df_facilities_train, df_facilities_test)
SVM_model3(df_facilities_train, df_facilities_test)
SVM_model4(df_facilities_train, df_facilities_test)
SVM_model5(df_facilities_train, df_facilities_test)
AdaBoost_model(df_facilities_train, df_facilities_test)

SVM baseline
Accuracy： 0.7857142857142857
Recall： 0.5864197530864198
Specificity： 0.9108527131782945
Precision： 0.8050847457627118
f1_score： 0.6785714285714286
AUC： 0.748636233132357


ADASYN
Accuracy： 0.8071428571428572
Recall： 0.7098765432098766
Specificity： 0.8682170542635659
Precision： 0.7718120805369127
f1_score： 0.7395498392282958
AUC： 0.7890467987367212


SMOTE
Accuracy： 0.8095238095238095
Recall： 0.6728395061728395
Specificity： 0.8953488372093024
Precision： 0.8014705882352942
f1_score： 0.7315436241610738
AUC： 0.7840941716910709


RandomOverSampler
Accuracy： 0.7976190476190477
Recall： 0.6666666666666666
Specificity： 0.8798449612403101
Precision： 0.7769784172661871
f1_score： 0.717607973421927
AUC： 0.7732558139534883


RandomUnderSampler
Accuracy： 0.7928571428571428
Recall： 0.7283950617283951
Specificity： 0.8333333333333334
Precision： 0.7329192546583851
f1_score： 0.7306501547987617
AUC： 0.7808641975308642


AdaBoost_model
Accuracy： 0.7142857142857143
Recall： 0.6234567901234568
Spe